In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample = pd.read_csv('/kaggle/input/sample/sample_labels.csv')

In [ ]:
sample2=sample[['Image Index','Finding Labels']]

In [ ]:
sample2.rename(columns={"Image Index": "imageName", "Finding Labels": "labels"},inplace=True)

In [ ]:
sample2

In [ ]:
for row in sample2.itertuples():
    if row.labels!="No Finding":
        sample2.iloc[row.Index]["labels"]="Found"
    else:
        sample2.iloc[row.Index]["labels"]="Not_Found"

In [ ]:
sample2

In [ ]:
import cv2
resized_images = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if not '.csv' in filename:
#             print(filename)
            file = cv2.imread(os.path.join(dirname, filename), cv2.IMREAD_GRAYSCALE)
            resized_images.append(cv2.resize(file, (512,512)))

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
plt.imshow(cv2.cvtColor(resized_images[0], cv2.COLOR_BGR2RGB)); plt.axis('off')

In [ ]:
resized_images[0]

In [ ]:
resized_images[3954]

In [ ]:
resized_images[3954].shape

In [ ]:
# import progressbar2
def feature_extraction(resized_images):
    alg = cv2.KAZE_create()
    features = []

    for image in resized_images:
        try:
            kps = alg.detect(image)
            kps = sorted(kps, key=lambda x: -x.response)
            kps, dsc = alg.compute(image, kps)
            dsc = dsc.flatten()
    #         print(type(dsc))

            features.append(dsc)
    #         print(dsc.shape)
            print(len(features))
        except Exception as e:
            print(e)
            print('image index: {}'.format(len(features)+1))
        
    return features

In [ ]:
features = feature_extraction(resized_images)

In [ ]:
# resized_images[0].shape
# resized_images[1].shape
print(resized_images[0])

In [ ]:
import pickle
with open('features.pkl', 'wb') as output:
    pickle.dump(features, output, pickle.HIGHEST_PROTOCOL)